In [ ]:
import ROOT
from ROOT import gROOT, TCanvas, TF1, TFile, TTree, gRandom, TH1F

from ROOT import RooRealVar, RooFormulaVar, RooVoigtian, RooChebychev, RooArgList, \
                 RooArgSet, RooAddPdf, RooDataSet, RooCategory, RooSimultaneous, \
                 RooBreitWigner, RooCBShape, RooFFTConvPdf, RooGaussian,RooExponential, \
                 RooBinning, kRed, kBlue, kDotted,TString,RooAbsData, RooPlot, TCut, RooAbsData, RooFit , RooStats,RooAbsReal              

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os, sys, time, random

from ROOT import TTree, TFile

# from root_numpy import root2array, rec2array, array2root

import pandas as pd
import numpy as np
import scipy 
import root_pandas as rp
import root_numpy as ry 

import pandas.core.common as com
from pandas.core.index import Index
from pandas.tools import plotting
from pandas.tools.plotting import scatter_matrix

from tqdm import tqdm_notebook

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.externals import joblib
from sklearn.metrics import classification_report, roc_curve, auc, roc_auc_score

sys.path.append('/home/chasenberg/repos/')
sys.path.append('/home/chasenberg/repos/dopy')
from dopy import * 
from dopy.dolearn.sklearn_utils import plot_roc_curve, plot_classifier_output, plot_correlations
from dopy.dolearn.sklearn_utils import plot_feature_importances, plot_classifier_output, classify_unseen_data
#from dopy.sklearn_utils import plot_bdt_vars
from dopy.doplot.plotting import Plotter, Plot
from dopy.doanalysis.df_utils import add_min_max, add_eta 

In [ ]:
sys.path.append('/home/chasenberg/repos/root_utils/')
from ROOT import TColor
import rootnotes  # For displaying TCanvas in notebooks
from lhcb_style import set_lhcb_style # Set basic LHCb plot style
set_lhcb_style()   
from root_utils import plot_pulls

In [ ]:
from ROOT import RooRealVar, RooExponential, RooExtendPdf, RooAddPdf, RooArgList, RooGaussian
from ROOT import RooFormulaVar
from ROOT import gSystem
gSystem.Load('libRooFit.so')
gSystem.Load('/home/chasenberg/repos/dopy/dolib/CustomShapes/libKll')
from ROOT import RooIpatia2

In [ ]:
mc_path = '/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/'
mc_file = 'random_sample.root'
mc_dir = os.path.join(mc_path, mc_file)

In [ ]:
cut_string_mc = 'idxPV==0'
tree_mc = 'Bd2JpsiKS'

In [ ]:
signal_dataframe_wrongPV  = rp.read_root(mc_dir,key=tree_mc,where=cut_string_mc, flatten=False)
signal_dataframe_wrongPV  = signal_dataframe_wrongPV.replace([np.inf, -np.inf], np.nan)
signal_dataframe_wrongPV  = signal_dataframe_wrongPV.dropna()

In [ ]:
sample = signal_dataframe_wrongPV.sample(122000)

In [ ]:
random_set = 'random_sample.root'
sample.to_root(mc_path+random_set,key='Bd2JpsiKS')

### Plot Delta_Tau

In [ ]:
signal_dataframe_wrongPV['Delta_TAU']
p = Plotter('/home/chasenberg/plots/selection/')
p.create_plots(signal_dataframe_wrongPV,'Delta_TAU',component_labels=['MC'])
p['Delta_TAU'].set_range(-0.0005,0.0005)
#bdt_plot.plot()
p.plot()

In [ ]:
#Mass variable
#mass = RooRealVar("B0_M", "B0_M", 5280, "MeV")
Delta_TAU = RooRealVar("Delta_TAU", "Delta_TAU", 0,-0.01,0.01, "ps")
idxPV = RooRealVar("idxPV", "Best PV", 0.,0., 0., "")

In [ ]:
#criteria and information for read in mc
data = ROOT.TFile(mc_dir)
tree_data = data.Get('Bd2JpsiKS') 
tree_data.GetEntries()

# QuadGaussian

In [ ]:
print(mc_path+random_set)

In [ ]:
#criteria and information for read in mc
data = ROOT.TFile(mc_path+'random_sample.root')
tree_data = data.Get('Bd2JpsiKS') 
tree_data.GetEntries()

In [ ]:
'''ntupleVarSet =  RooArgSet(Delta_TAU,idxPV)
dataset = RooDataSet('data','data',tree_data,ntupleVarSet)
ROOT.SetOwnership(dataset,False)'''

## Build per event resolution model 


## Quad Gaussian

# Variables 

In [ ]:
from ROOT import RooGaussian, RooAddPdf

obsTime = RooRealVar("B0_FitPVConst_tau", "obsTime", 0,-0.001,0.001, "ps")
obsTimeErr = RooRealVar("B0_FitPVConst_tauErr", "obsTimeErr", 0,-0.001,0.001, "ps")
obsTimeRes = RooRealVar("Delta_TAU", "obsTimeRes", 0,-0.001,0.001, "ps")

observables = RooArgSet(obsTimeRes, obsTimeErr)
dataset = RooDataSet("data", "data", tree_data, observables)
ROOT.SetOwnership(dataset,False)

parResCondMean  = RooRealVar('parResCondMean', 'parResCondMean', 0.0, -5.0, 5.0)
ROOT.SetOwnership(parResCondMean, False)

parResCond_a_1 = RooRealVar('parResCond_a_1', 'parResCond_a_1', 4.0, 0.1, 10)
ROOT.SetOwnership(parResCond_a_1, False)
parResCond_b_1 = RooRealVar('parResCond_b_1', 'parResCond_b_1', 0.02, -0.0000005, 0.5)
ROOT.SetOwnership(parResCond_b_1, False)


parResCond_a_2 = RooRealVar('parResCond_a_2', 'parResCond_a_2',25, 0.0, 30)
ROOT.SetOwnership(parResCond_a_2, False)
parResCond_b_2 = RooRealVar('parResCond_b_2', 'parResCond_b_2', 0.03, 0., 0.5)
ROOT.SetOwnership(parResCond_b_2, False)



parResCond_a_3 = RooRealVar('parResCond_a_3', 'parResCond_a_3', 0.01, 0.0001, 0.06)
ROOT.SetOwnership(parResCond_a_3, False)
parResCond_b_3 = RooRealVar('parResCond_b_3', 'parResCond_b_3', 0.1, 0, 0.5)
ROOT.SetOwnership(parResCond_b_3, False)


parResCond_a_4 = RooRealVar('parResCond_a_4', 'parResCond_a_4', 0.0001,0.0001,0.0001)
ROOT.SetOwnership(parResCond_a_4, False)
parResCond_b_4 = RooRealVar('parResCond_b_4', 'parResCond_b_4', 0.5, 0.0,8)
ROOT.SetOwnership(parResCond_b_4, False)


#Definition and calculation of sigma's, related linearly on the decaytime error from the ntuple 
parResCondSigma_1 = RooFormulaVar('parResCondSigma_1', '@0*@1+@2', RooArgList(parResCond_a_1, obsTimeErr, parResCond_b_1))
ROOT.SetOwnership(parResCondSigma_1, False)

parResCondSigma_2 = RooFormulaVar('parResCondSigma_2', '@0*@1+@2', RooArgList(parResCond_a_2, obsTimeErr, parResCond_b_2))
ROOT.SetOwnership(parResCondSigma_2, False)

parResCondSigma_3 = RooFormulaVar('parResCondSigma_3', '@0*@1+@2', RooArgList(parResCond_a_3, obsTimeErr, parResCond_b_3))
ROOT.SetOwnership(parResCondSigma_3, False)

parResCondSigma_4 = RooFormulaVar('parResCondSigma_4', '@0*@1+@2', RooArgList(parResCond_a_4, obsTimeErr, parResCond_b_4))
ROOT.SetOwnership(parResCondSigma_4, False)

#Fractions of particular PDF's  
parResCondFrac_1 = RooRealVar('parResCondFrac_1', 'parResCondFrac_1', 0.5, 0.01, 0.6)
ROOT.SetOwnership(parResCondFrac_1, False)
parResCondFrac_2 = RooRealVar('parResCondFrac_2', 'parResCondFrac_2', 0.48, 0.01, 0.5)
ROOT.SetOwnership(parResCondFrac_2, False)
parResCondFrac_3 = RooRealVar('parResCondFrac_3', 'parResCondFrac_3', 0.05, 0.000001, 0.5)
ROOT.SetOwnership(parResCondFrac_3, False)


#Defition of the four Gaussians     
pdfGaussianCond_1 = RooGaussian('pdfGaussianCond_1', 'pdfGaussianCond_1', obsTimeRes, parResCondMean, parResCondSigma_1)
ROOT.SetOwnership(pdfGaussianCond_1, False)
    
pdfGaussianCond_2 = RooGaussian('pdfGaussianCond_2', 'pdfGaussianCond_2', obsTimeRes, parResCondMean, parResCondSigma_2)
ROOT.SetOwnership(pdfGaussianCond_2, False)
    
pdfGaussianCond_3 = RooGaussian('pdfGaussianCond_3', 'pdfGaussianCond_3', obsTimeRes, parResCondMean, parResCondSigma_3)
ROOT.SetOwnership(pdfGaussianCond_3, False)

pdfGaussianCond_4 = RooGaussian('pdfGaussianCond_4', 'pdfGaussianCond_4', obsTimeRes, parResCondMean, parResCondSigma_4)
ROOT.SetOwnership(pdfGaussianCond_4, False)


#Use yields instead of fractions 
yield_1 = RooRealVar("yield_1", "yield_1", 56745, 55000,70000)
yield_2 = RooRealVar("yield_2", "yield_2", 58406, 20000,60000)
yield_3 = RooRealVar("yield_3", "yield_3", 10000, 1,20000)
yield_4 = RooRealVar("yield_4", "yield_4", 1000, 10,10000)


# including four components -> 4 Gaussians
pdfGaussianCond_sum = RooAddPdf('pdfGaussianCond_sum', 'pdfGaussianCond_sum', RooArgList(pdfGaussianCond_1, pdfGaussianCond_2, pdfGaussianCond_3, pdfGaussianCond_4  ), RooArgList(parResCondFrac_1,parResCondFrac_2,parResCondFrac_3))
ROOT.SetOwnership(pdfGaussianCond_sum, False)


data_model_cond = pdfGaussianCond_sum
data_model_cond.Print('t')

In [ ]:
results_dir = '/home/chasenberg/plots/resolution/results/'

parameters_file_name  = 'params_cond.txt'
parameters_init_name  = parameters_file_name.replace('.txt', '_init.txt')
parameters_final_name = parameters_file_name.replace('.txt', '_final.txt')

if os.path.exists(os.path.join(results_dir, parameters_init_name)):
    print('INFO: Reading parameters from configfile '  + os.path.join(results_dir, parameters_init_name), flush=True)
    data_model_cond.getParameters(data).readFromFile(os.path.join(results_dir, parameters_init_name))
else:
    print('INFO: configfile... '  + os.path.join(results_dir, parameters_init_name) + ' does not exist', flush=True)
    
cond_argset = RooArgSet(obsTimeErr)    
cond_argset.remove(obsTimeErr, True, True)

fit_result = data_model_cond.fitTo(dataset, 
                                    RooFit.NumCPU(6),
                                    RooFit.Minimizer('Minuit','minimize'),
                                    RooFit.Hesse(True),
                                    RooFit.Save(True))#,
                                    #RooFit.ConditionalObservables(cond_argset)) 

if fit_result:
    fit_result.Print('v')
    fit_result.correlationMatrix().Print('v')
    if fit_result.status()==0:
        print('INFO: Fit converged')
    else:
        print('INFO: Status '+str(fit_result.status()))
    print('INFO: Fit complete', flush=True)
else:
    print('ERROR: There is no fitresult', flush=True)
 
final_params_file = os.path.join(results_dir, parameters_final_name)
data_model_cond.getParameters(dataset).writeToFile(final_params_file)

In [ ]:
print(parResCondFrac_1.getVal()),print(parResCondFrac_2.getVal()),print(parResCondFrac_3.getVal()) 

In [ ]:
print(parResCond_a_1.getVal()),print(parResCond_a_2.getVal()),print(parResCond_a_3.getVal()),print(parResCond_a_4.getVal()) 

In [ ]:
print(parResCond_b_1.getVal()),print(parResCond_b_2.getVal()),print(parResCond_b_3.getVal()),print(parResCond_b_4.getVal()) 

## Plot not like a pro

In [ ]:
# Prepare frame
%matplotlib inline  
import ROOT
import sys
sys.path.append('/home/chasenberg/repos/root_utils/')
from ROOT import TColor
import rootnotes  # For displaying TCanvas in notebooks
from lhcb_style import set_lhcb_style # Set basic LHCb plot style

from root_utils import plot_pulls

frame = Delta_TAU.frame(ROOT.RooFit.Bins(180))

dataset.plotOn(frame, ROOT.RooFit.Name("data1"))

#model.plotOn( frame , ROOT.RooFit.Components("signal"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(kRed))

data_model_cond.plotOn( frame , ROOT.RooFit.Components("pdfGaussianCond_1"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(400))#
data_model_cond.plotOn( frame , ROOT.RooFit.Components("pdfGaussianCond_2"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(kBlue))#model.plotOn( frame , ROOT.RooFit.Components("signal_2"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(416))

data_model_cond.plotOn( frame , ROOT.RooFit.Components("pdfGaussianCond_3"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(400))#
#data_model_cond.plotOn( frame , ROOT.RooFit.Components("pdfGaussianCond_4"), ROOT.RooFit.LineStyle(kDotted), ROOT.RooFit.LineColor(kBlue))#
data_model_cond.plotOn(frame, ROOT.RooFit.Name("fit"))
# Add additional informations to the plot
text_size = 0.035
# Create TLegend
legend = ROOT.TLegend(0.7, 0.75, 0.9, 0.9, '')
legend.AddEntry(frame.findObject('data1'), "Data points", "p");
legend.AddEntry(frame.findObject('fit'), 'Fit', 'l')
legend.SetTextSize(text_size)

# Create TLatex
'''latex = ROOT.TLatex(0.19, 0.8, '#splitline{Toy MC example}' + \
                                 '{#it{f}(x) = #frac{1}{#sigma #sqrt{2 #pi}}' + \
                                 ' e^{- 0.5 #left( #frac{x - #mu}{#sigma} #right)^{2}}}')'''
#latex.SetTextSize(text_size)

# Plot pulls
can, _ = plot_pulls('test', frame, legend=legend,logy=True)#, latex=latex)
can.SaveAs("/home/chasenberg/plots/resolution/average_resolution.pdf")
can  # To display plot in notebookspdfGaussianCond_1